Step 1 convert fits to csv

In [5]:
import numpy
import sys
from astropy.io import fits

def write_nparr_to_csv(data, filename, delimiter):
    # Ajout d'une colonne vide à droite
    empty_column = numpy.full((data.shape[0], 1), numpy.nan)  # Colonne remplie de NaN
    data = numpy.hstack((data, empty_column))

    # Écriture dans le fichier CSV/TSV
    numpy.savetxt(filename, data, delimiter=delimiter, fmt='%s')

# Lecture du fichier FITS
gt_name = "SGRA_full_gt.fits"
gt_hdu = fits.open(gt_name)[0]
data = gt_hdu.data

# Si le tableau a plus de 2 dimensions, sélectionne la première tranche
if data.ndim > 2:
    data = data[0, 0, :, :]

# Réinversion du tableau (inversion des axes)
result = numpy.flip(data)

# Écriture dans le fichier CSV/TSV
IMAGE = "image.csv"
write_nparr_to_csv(result, IMAGE, ",")
print("Fichier suvegarder dans : "+IMAGE)


Fichier suvegarder dans : image.csv


Step2 run the dataflow executable

In [ ]:
EXECUTABLE = "./STD_DEGRID"
IMAGE = "image.csv"
command = [EXECUTABLE, IMAGE]
print(f"Lancement : {' '.join(command)}")
result = subprocess.run(command, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print(f"Erreur :\n{result.stderr}")